# Materials
* [Bunch of articles](http://www.mitpressjournals.org/doi/pdf/10.1162/neco.1993.5.6.954) - I strongly recomment this resource, cause it hosts most actual (by year of publishing) articles.
* [realization on C++](https://github.com/BelBES/ESOINN)
* [ESOINN algorithm](http://cs.nju.edu.cn/rinc/SOINN/e-soinn.pdf)
* [Detailed article](http://www.haselab.info/soinn-e.html)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import random


First, let's determine the abstract class for *neural network* (NN), which implements the base NN structure with voluntary neuron's types.

In [6]:
class ESOINN_Rib:
    def __init__(self):
        self.rib_exist = 0
        self.rib_age = 0

In [7]:
class ESOINN_Neuron:
    def __init__(self, feature_vector=()):
        self.feature_vector = feature_vector  
        self.accamulate_signals = 0
        self.total_points = 0
        self.density = 0
        self.subclass_id = -1

In [ ]:
class ESOINN:
    def __init__(self, init_neurons, C1=0.001, C2=1, learning_step=200, max_age=50):
        self.C1 = C1
        self.C2 = C2
        self.learning_step = learning_step
        self.max_age
        self.signals_amount = 0
        
        self.neurons = np.array([
            ESOINN_Neuron(init_neurons[0]),
            ESOINN_Neuron(init_neurons[1])
        ], dtype=ESOINN_Neuron)
        
        self.adjacency_matrix = np.array([[ESOINN_Rib()]*2]*2, dtype=ESOINN_Rib)
        
        self.winners_indexes = []      

array([['str', 'str'],
       ['str', 'str']], 
      dtype='<U3')